In [1]:
import lightgbm as lgb
from sklearn.grid_search import GridSearchCV
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [14]:
import pandas as pd
data_tree_train = pd.read_csv('data_tree_train_final.csv',encoding='gbk')
data_tree_test = pd.read_csv('data_tree_test_final.csv',encoding='gbk')

data_tree_train = data_tree_train.fillna(0)
data_tree_test = data_tree_test.fillna(0)

data_tree_train_1 = data_tree_train[data_tree_train['label']==1]
data_tree_train_0 = data_tree_train[data_tree_train['label']==0]

train0,test0 = train_test_split(data_tree_train_0, test_size=0.1)
train1,test1 = train_test_split(data_tree_train_1, test_size=0.1)

test = pd.concat([test0,test1],ignore_index=True)

In [3]:
train1Plus = train1.sample(13000,replace=True)
All = pd.concat([train0,train1Plus],ignore_index=True)

In [4]:
All.label.value_counts()

0    17100
1    13000
Name: label, dtype: int64

In [5]:
train1Plus = data_tree_train_1.sample(14000,replace=True)
train = pd.concat([data_tree_train_0,train1Plus],ignore_index=True)
train.label.value_counts()

0    19000
1    14000
Name: label, dtype: int64

In [6]:
pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_columns', 1000)
target='label'
IDcol = '个人编码'
predictors = [x for x in data_tree_train.columns if x not in [target, IDcol]]

In [16]:
lgb2 = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=10, max_depth=7, 
        learning_rate=0.1, n_estimators=256, max_bin=255, subsample_for_bin=50000, 
        min_split_gain=0, min_child_weight=1, min_child_samples=10, 
        subsample=1, subsample_freq=1, colsample_bytree=1, reg_alpha=0, reg_lambda=0, 
        seed=0, nthread=-1, silent=True)
lgb2.fit(train[predictors], train['label'])

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=7,
        max_drop=50, min_child_samples=10, min_child_weight=1,
        min_split_gain=0, n_estimators=256, nthread=-1, num_leaves=10,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [17]:
feaImp = pd.Series(lgb2.booster().feature_importance()).sort_values(ascending=False)#获得重要性不为0的特征
goodFeature = feaImp[feaImp>0].index
features = data_tree_train_0[predictors].columns[goodFeature]
features

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function booster is deprecated; Use attribute booster_ instead.
  warnings.warn(msg, category=DeprecationWarning)


Index(['单个顺序号数量_sum', '本次审批金额_sum', '当月就诊次数max', '当月就诊次数var', '药品费发生金额_sum',
       '基本医疗保险统筹基金支付金额_sum', '单个顺序号数量_mean', '三目统计项目_1.0_sum', '非账户支付金额_max',
       '基本医疗保险统筹基金支付金额_max',
       ...
       '医院编码_1015.0', '手术费发生金额_mean', '医院编码_192.0', '医院编码_925.0',
       '操作时间_dayofweek_6.0_sum', '床位费发生金额_var', '检查费申报金额_max',
       '交易时间_dayofweek_6.0_sum', '当天就诊次数min', '医院编码_1276.0'],
      dtype='object', length=347)

In [15]:
start = time.time()
param_test1 = {
#     'num_leaves':np.array([10,32,100]),
#     'max_depth':np.array([5,10,-1]),
#     'min_child_weight':np.array([1,3,5,10]),
#     'gamma':[i/10.0 for i in range(0,5)],
#     'n_estimators':np.array([100,500,1000])
}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(n_estimators=200, max_features=0.5), 
                       param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=8)
gsearch1.fit(data_tree_train[predictors],data_tree_train[target])
end = time.time()
print('运行时间：')
(end-start)/60

运行时间：


106.14810781081518

In [18]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.46191, std: 0.03916, params: {}], {}, 0.4619087100055571)

In [ ]:
start = time.time()
param_test1 = {
#     'num_leaves':np.array([10,32,100]),
#     'max_depth':np.array([5,10,-1]),
#     'min_child_weight':np.array([1,3,5,10]),
#     'gamma':[i/10.0 for i in range(0,5)],
#     'n_estimators':np.array([100,500,1000])
}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators=200, max_features=0.5), 
                       param_grid = param_test1, scoring='f1',n_jobs=4,iid=False, cv=8)
gsearch2.fit(data_tree_train[features],data_tree_train[target])
end = time.time()
print('运行时间：')
(end-start)/60

In [ ]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_